In [15]:
import pandas as pd
from bs4 import BeautifulSoup

with open(r"C:\Users\s3309\Desktop\Investment\Material\tifrs-fr1-m1-ci-cr-2330-2022Q2.html", encoding="UTF-8") as f:
    data=f.read()
soup = BeautifulSoup(data,'html.parser')


adsheet=soup.find_all('table')[0]
time_window=[]
for i in adsheet.find_all('tr')[1].find_all('th')[2:]:
    time_window.append(i.find(class_='en').text)


cate=adsheet.find_all('tr')[2].find_all('td')[1].find(class_='en').text
sub_cate=adsheet.find_all('tr')[3].find_all('td')[1].find(class_='en').text


In [16]:
sub_cate

'\u3000Current assets'